# Import the basic python libraries

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib
%matplotlib inline
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')
sns.set(style='white', context='notebook', palette='deep')

import glob
from datetime import datetime

import sklearn #A Machine Learning library
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')



# Read the datasets

In [10]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# Non ho specificato alcuna cartella assoluta in quanto i 
#csv si trovano nella stessa cartella del notebook di training
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70736 entries, 0 to 70735
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   No-show         70736 non-null  object 
 1   PatientId       70736 non-null  float64
 2   AppointmentID   70736 non-null  int64  
 3   Gender          70736 non-null  object 
 4   ScheduledDay    70736 non-null  object 
 5   AppointmentDay  70736 non-null  object 
 6   Age             70736 non-null  int64  
 7   Neighbourhood   70736 non-null  object 
 8   Scholarship     70736 non-null  int64  
 9   Hipertension    70736 non-null  int64  
 10  Diabetes        70736 non-null  int64  
 11  Alcoholism      70736 non-null  int64  
 12  Handcap         70736 non-null  int64  
 13  SMS_received    70736 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 7.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17685 entries, 0 to 17684
Data columns (total 14 columns):


# Exploratory data analysis

### Describe data

In [138]:
train.describe()



,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,7.073600e+04,7.073600e+04,70736.000000,70736.000000,70736.000000,70736.000000,70736.000000,70736.00000,70736.000000
mean,1.478240e+14,5.675431e+06,37.171384,0.099553,0.198386,0.072551,0.030480,0.02228,0.321873
std,2.560265e+14,7.132269e+04,23.067551,0.299406,0.398787,0.259401,0.171904,0.16324,0.467198
min,3.921784e+04,5.030230e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,4.168386e+12,5.640464e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,3.232918e+13,5.680778e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,9.464947e+13,5.725629e+06,55.250000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000
max,9.999816e+14,5.790481e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.00000,1.000000


In [140]:
test.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.768500e+04,1.768500e+04,17685.000000,17685.000000,17685.000000,17685.000000,17685.000000,17685.000000,17685.000000
mean,1.511027e+14,5.675234e+06,36.937631,0.097936,0.196777,0.071416,0.031269,0.022844,0.320271
std,2.612568e+14,7.136192e+04,23.250527,0.297237,0.397574,0.257526,0.174050,0.159657,0.466594
min,4.374176e+04,5.134197e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.164541e+12,5.640550e+06,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.985670e+13,5.680669e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.458256e+13,5.725225e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999350e+14,5.790376e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [141]:
train.head(7)

,No-show,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,No,8.989312e+13,5670134,F,2016-05-06T13:06:20Z,2016-05-10T00:00:00Z,63,MONTE BELO,0,0,0,0,0,1
1,Yes,2.971169e+13,5652716,F,2016-05-03T10:31:51Z,2016-05-09T00:00:00Z,31,SANTOS REIS,0,0,1,0,0,0
2,No,5.414642e+12,5690801,F,2016-05-12T10:41:47Z,2016-05-12T00:00:00Z,71,JARDIM CAMBURI,0,0,0,0,0,0
3,Yes,5.258853e+14,5653220,F,2016-05-03T11:36:42Z,2016-05-18T00:00:00Z,0,ILHA DE SANTA MARIA,0,0,0,0,0,0
4,No,4.924131e+12,5782097,M,2016-06-07T10:34:15Z,2016-06-07T00:00:00Z,41,DO QUADRO,0,0,0,0,0,0
5,No,5.727532e+12,5648722,M,2016-05-02T16:25:48Z,2016-05-05T00:00:00Z,48,GRANDE VITÓRIA,0,0,0,0,0,0
6,No,2.821425e+14,5731929,M,2016-05-24T10:43:55Z,2016-06-03T00:00:00Z,26,PARQUE MOSCOSO,0,0,0,0,0,0


In [40]:
test.head(3)

,Unnamed: 0,No-show,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,55824,No,8.648938e+12,5707284,M,2016-05-17T09:27:12Z,2016-05-20T00:00:00Z,37,SANTO ANTÔNIO,0,0,0,0,0,0
1,46525,Yes,2.535634e+12,5693684,F,2016-05-13T07:34:07Z,2016-05-18T00:00:00Z,48,RESISTÊNCIA,0,1,0,0,0,0
2,6160,No,9.967568e+13,5765655,M,2016-06-02T13:29:04Z,2016-06-02T00:00:00Z,50,GURIGICA,0,0,0,0,0,0


#### Number of Rows and Columns

In [204]:
train.shape

(70736, 16)

In [205]:
test.shape

(17685, 16)

#### Column Names

In [206]:
train.columns

Index(['No-show', 'PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'Month',
       'AgeClass'],
      dtype='object')

In [207]:
test.columns

Index(['No-show', 'PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'Month',
       'AgeClass'],
      dtype='object')

#### Target Variable

In [11]:
no_show = train["No-show"].value_counts()
print(no_show)

Percent_no_show = no_show["Yes"]/ no_show.sum() * 100
print("Percent who didn't show up to their appointment:",Percent_no_show )

No     56453
Yes    14283
Name: No-show, dtype: int64
Percent who didn't show up to their appointment: 20.191981452160146


In [12]:
#Checking if there are are any missing values
train.isnull().sum()

No-show           0
PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
dtype: int64

In [144]:
print (train.isnull())

       No-show  PatientId  AppointmentID  Gender  ScheduledDay  \
0        False      False          False   False         False   
1        False      False          False   False         False   
2        False      False          False   False         False   
3        False      False          False   False         False   
4        False      False          False   False         False   
...        ...        ...            ...     ...           ...   
70731    False      False          False   False         False   
70732    False      False          False   False         False   
70733    False      False          False   False         False   
70734    False      False          False   False         False   
70735    False      False          False   False         False   

       AppointmentDay    Age  Neighbourhood  Scholarship  Hipertension  \
0               False  False          False        False         False   
1               False  False          False        False   

#### Recode Target Variable

In [13]:
train['No-show'].replace("No", 0,inplace=True)
train['No-show'].replace("Yes", 1,inplace=True)

#### Exploratory Analysis on Features
#### Count of Male vs Female

In [14]:
train["Gender"].value_counts()

F    46008
M    24728
Name: Gender, dtype: int64

#### Seeing how each feature relates to the target variable.

In [15]:
Exploratory_Analysis = ['Gender','Hipertension','Alcoholism','Diabetes']
for r in Exploratory_Analysis :
    print(train.groupby(r)['No-show'].mean())
#train.groupby('Hipertension')['No-show'].mean()

Gender
F    0.203508
M    0.198965
Name: No-show, dtype: float64
Hipertension
0    0.208784
1    0.174320
Name: No-show, dtype: float64
Alcoholism
0    0.202167
1    0.194058
Name: No-show, dtype: float64
Diabetes
0    0.203725
1    0.178781
Name: No-show, dtype: float64


##### Come mostrato sopra, il genere non sembra influire sul fatto che una persona si presenti o meno a un appuntamento, poiché sia i maschi che le femmine non si presentano a circa il 20 percento dei loro appuntamenti.
#### Anche il diabete e l'alcolismo non sembrano influenzare con la mancata presenza all'appuntamento.

### Whether the patient received a text reminder

In [16]:
train.groupby('SMS_received')['No-show'].mean()

SMS_received
0    0.166008
1    0.278082
Name: No-show, dtype: float64

#### Inaspettatamente, il 27% dei pazienti che hanno ricevuto un sms  non si sono presentati ai loro appuntamenti, mentre il 17% dei pazienti che non hanno ricevuto un sms si sono presentati ai loro appuntamenti

#### Feature Engineering

##### Age

In [17]:
min_age = train.Age.min()
print("Min Age:", min_age)
max_age = train.Age.max()
print("Max Age:", max_age)

Min Age: 0
Max Age: 115


### Prima di procedere con la costruzione del modello elimino dal trainining set  i valori anomali in base all'età

In [18]:
train = train[(train.Age >= 0) & (train.Age <= 100)] # in questo modo elimino gli outlier

#### Wait Time

##### Se pianifico un appuntamento con largo anticipo e nessuno me lo ricorda, tendo a dimenticarmene. O quando arriva il giorno dell'appuntamento, mi sento meglio e non mi preoccupo di presentarmi. In questa sezione, creerò una variabile chiamata "wait_time" per vedere se il tempo che intercorre tra la data in cui è stato programmato un appuntamento e la data in cui è stato fissato l'appuntamento ha un effetto sul non presentarsi all'appuntamento.

In [19]:
import numpy as np

# Converts the two variables to datetime variables
train['ScheduledDay'] = pd.to_datetime(train['ScheduledDay'])
train['AppointmentDay'] = pd.to_datetime(train['AppointmentDay'])

# Create a variable called "AwaitingTime" by subtracting the date the patient made the appointment and the date of the appointment.
train['AwaitingTime'] = train["AppointmentDay"].sub(train["ScheduledDay"], axis=0)

# Convert the result "AwaitingTime" to number of days between appointment day and scheduled day. 
train["AwaitingTime"] = (train["AwaitingTime"] / np.timedelta64(1, 'D')).abs()

## Number of Appointments Missed in the Past

#### Ho creato una nuova variabile che aggiunge il numero di appuntamenti che il paziente ha mancato in passato.

In [20]:
# Number of Appointments Missed by Patient
train['Num_App_Missed'] = train.groupby('PatientId')['No-show'].apply(lambda x: x.cumsum())

In [21]:
train.columns

Index(['No-show', 'PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'AwaitingTime',
       'Num_App_Missed'],
      dtype='object')

#### Lascerò delle  colonne che non verranno utilizzate per la modellazione

In [22]:
train.drop(["PatientId",'AppointmentID',
         'ScheduledDay','Handcap','AppointmentDay','Neighbourhood'], axis=1, inplace = True)

#### Di seguito le features finali e le variabili target 

In [23]:
X = train[['Gender', 'Diabetes','Hipertension', 'Scholarship', 'SMS_received',
         'Num_App_Missed', 'Age', 'AwaitingTime']]

y = train["No-show"]


In [24]:
#This will create dummies for the remaining variables (Gender)
X_train1 = pd.get_dummies(X)

#### Questo approccio è più flessibile (rispetto al Label Encoding e al One-Hot Encoding), perché consente di codificare tutte le colonne (features) desiderate e di scegliere come etichettare le colonne utilizzando un prefisso. 

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train1)
rescaledX2 = scaler.transform(X_train1)
# summarize transformed data
np.set_printoptions(precision=3)
print(rescaledX2[0:5,:])

[[-0.279 -0.499 -0.333 -0.687 -0.519 -1.26  -0.553 -1.364  1.364]
 [-0.279  2.005 -0.333 -0.687 -0.519  1.034 -0.419  0.733 -0.733]
 [-0.279 -0.499 -0.333  1.456  1.003 -1.433 -0.235  0.733 -0.733]
 [-0.279 -0.499 -0.333 -0.687  1.003 -1.433  0.093  0.733 -0.733]
 [-0.279 -0.499 -0.333  1.456 -0.519 -0.481 -0.233  0.733 -0.733]]


In [29]:

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Create Training and Test Dataset with 75% Training and 25% Test
X_train, X_test, y_train, y_test = train_test_split(rescaledX2, y, test_size=0.25)

# Run Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

# Analyze results
print("Results:")
print("Accuracy", metrics.accuracy_score(y_test,y_pred))

# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

#Specificity: When the actual value is negative, how often is the prediction correct?
print("Specificity:",TN / float(TN + FP))

#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print("False Positive Rate:",FP / float(TN + FP))



Results:
Accuracy 0.917943787818809
Specificity: 0.9378010767922924
False Positive Rate: 0.06219892320770756


In [30]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  GradientBoostingClassifier

# Create Training and Test Dataset with 75% Training and 25% Test
X_train, X_test, y_train, y_test = train_test_split(rescaledX2, y, test_size=0.25)

# Run GradientBoostingClassifier
grad = GradientBoostingClassifier()
grad.fit(X_train, y_train)
y_pred = grad.predict(X_test)

# Analyze results
print("Results:")
print("Accuracy", metrics.accuracy_score(y_test,y_pred))

# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

#Specificity: When the actual value is negative, how often is the prediction correct?
print("Specificity:",TN / float(TN + FP))

#False Positive Rate: When the actual value is negative, how often is the prediction incorrect?
print("False Positive Rate:",FP / float(TN + FP))


Results:
Accuracy 0.9327602782333314
Specificity: 0.9410973451327433
False Positive Rate: 0.05890265486725664
